In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import ipywidgets as widget
%matplotlib inline

conn = sqlite3.connect('data/old_123database.db')

In [2]:
c = conn.cursor()
cursor = c.execute('SELECT dataset.name, dataset.id FROM dataset ORDER BY dataset.name')

cursor.fetchall()



[('kopenhagen', 1)]

In [3]:
c = conn.cursor()
cursor = c.execute('SELECT * FROM actor WHERE actor.id = 2')
cursor.fetchall()


[(2, 'euinclnorway', 'euinclnorway', 1)]

In [8]:
c = conn.cursor()
cursor = c.execute('SELECT * FROM modelrun')
cursor.fetchall()

[(1, '2018-02-12 12:58:18.391279', '2018-02-12 12:59:34.843931', 0, 10, 10, 1),
 (2, '2018-02-12 13:30:09.394770', '2018-02-12 13:41:48.245693', 0, 10, 90, 1),
 (3, '2018-02-12 14:22:56.271112', None, 0, 10, 100, 1),
 (4, '2018-02-12 14:23:12.300890', None, 0, 10, 100, 1)]

In [11]:
df = pd.read_sql("""
SELECT
  r.pointer as repetition,
  i.pointer as iteration,
  a.name as actor,
  demand_actor.name as j,
  supply.name as supply,
  demand.name as demand,
  ea.x,
  ea.y,
  ea.eu,
  ea.demand_position
  FROM exchangeactor ea
    LEFT JOIN actor a ON a.id = ea.actor_id
    LEFT JOIN issue supply ON ea.supply_issue_id = supply.id
    LEFT JOIN issue demand ON ea.demand_issue_id = demand.id
    LEFT JOIN exchange e ON ea.id = e.j_id or ea.id = e.i_id
    LEFT JOIN actor demand_actor ON demand_actor.id = ea.other_actor_id
    LEFT JOIN iteration i ON e.iteration_id = i.id
    LEFT JOIN repetition r ON i.repetition_id = r.id
    LEFT JOIN modelrun m ON r.model_run_id = m.id
  WHERE ea.actor_id = 2 AND i.pointer = 9 AND m.id = 1
        AND (ea.supply_issue_id = 1 or ea.demand_issue_id = 1)
  ORDER BY e.id ASC;
""", 
                 conn
                )

df



,repetition,iteration,actor,j,supply,demand,x,y,eu,demand_position
0,0,9,euinclnorway,chinaindia,Newtreaty,Control,39.999812,96.837238,1.156015,96.837238
1,1,9,euinclnorway,usa,Newtreaty,Domestred,62.257279,9.359465,1.166123,9.359465
2,2,9,euinclnorway,usa,Newtreaty,Control,62.756292,14.857080,1.255663,11.579552
3,2,9,euinclnorway,usa,Newtreaty,MRVdecrease,14.857080,11.579552,0.008888,11.579552
4,3,9,euinclnorway,japan,Newtreaty,Domestred,61.960925,44.623149,0.241805,44.623149
5,3,9,euinclnorway,usa,Newtreaty,MRVdecrease,44.623149,12.443919,0.087266,12.443919
6,4,9,euinclnorway,chinaindia,Newtreaty,Domestred,41.643271,95.941451,0.935466,95.941451
7,5,9,euinclnorway,usa,Newtreaty,Control,60.954195,12.440327,1.271776,8.790738
8,5,9,euinclnorway,usa,Newtreaty,MRVdecrease,12.440327,8.790738,0.009897,8.790738
9,6,9,euinclnorway,chinaindia,Newtreaty,Domestred,42.133168,97.790103,0.958875,97.790103


In [12]:
pd.pivot_table(df, index=['repetition', 'j','demand', 'supply'])

demand_position        eu  \
repetition j          demand      supply                                 
0          chinaindia Control     Newtreaty        96.837238  1.156015   
1          usa        Domestred   Newtreaty         9.359465  1.166123   
2          usa        Control     Newtreaty        11.579552  1.255663   
                      MRVdecrease Newtreaty        11.579552  0.008888   
3          japan      Domestred   Newtreaty        44.623149  0.241805   
           usa        MRVdecrease Newtreaty        12.443919  0.087266   
4          chinaindia Domestred   Newtreaty        95.941451  0.935466   
5          usa        Control     Newtreaty         8.790738  1.271776   
                      MRVdecrease Newtreaty         8.790738  0.009897   
6          chinaindia Domestred   Newtreaty        97.790103  0.958875   
7          usa        Control     Newtreaty        10.913069  1.347381   
8          japan      Domestred   Newtreaty        45.404210  0.249816   
           usa        MRVdecrease Newtreaty        12.866285  0.082615   
9          chinaindia Domestred   Newtreaty        97.836671  1.022138   

                                             iteration          x          y  
repetition j          demand      supply                                      
0          chinaindia Control     Newtreaty          9  39.999812  96.837238  
1          usa        Domestred   Newtreaty          9  62.257279   9.359465  
2          usa        Control     Newtreaty          9  62.756292  14.857080  
                      MRVdecrease Newtreaty          9  14.857080  11.579552  
3          japan      Domestred   Newtreaty          9  61.960925  44.623149  
           usa        MRVdecrease Newtreaty          9  44.623149  12.443919  
4          chinaindia Domestred   Newtreaty          9  41.643271  95.941451  
5          usa        Control     Newtreaty          9  60.954195  12.440327  
                      MRVdecrease Newtreaty          9  12.440327   8.790738  
6          chinaindia Domestred   Newtreaty          9  42.133168  97.790103  
7          usa        Control     Newtreaty          9  62.310993  10.913069  
8          japan      Domestred   Newtreaty          9  63.316329  45.404210  
           usa        MRVdecrease Newtreaty          9  45.404210  12.866285  
9          chinaindia Domestred   Newtreaty          9  38.507688  97.836671

In [14]:
df = pd.read_sql("""SELECT a.name as actor, i.name as issue, ai.position FROM actorissue AS ai
  LEFT JOIN actor a ON ai.actor_id = a.id
  LEFT JOIN issue i ON ai.issue_id = i.id
  LEFT JOIN iteration i2 ON ai.iteration_id = i2.id
  LEFT JOIN repetition r ON i2.repetition_id = r.id
  WHERE i.id = 1 AND a.id = 2 AND ai.type = 'before' AND i2.pointer = 9;
  """, conn)
df


,actor,issue,position
0,euinclnorway,Newtreaty,39.999812
1,euinclnorway,Newtreaty,62.257279
2,euinclnorway,Newtreaty,62.756292
3,euinclnorway,Newtreaty,61.960925
4,euinclnorway,Newtreaty,41.643271
5,euinclnorway,Newtreaty,60.954195
6,euinclnorway,Newtreaty,42.133168
7,euinclnorway,Newtreaty,62.310993
8,euinclnorway,Newtreaty,63.316329
9,euinclnorway,Newtreaty,38.507688


In [15]:
table = pd.pivot_table(df, index=['actor'], columns=['issue'],aggfunc=[np.std, np.average])
table

,std,average
,position,position
issue,Newtreaty,Newtreaty
actor,,
euinclnorway,11.079023,51.723172
